LSTM trained on gridded forcings for each station, one model for all stations

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, datasets, utils

time_stamp = '20190905-102708'
time_stamp

'20190905-102708'

In [2]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
print('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)

cuda devices: []


In [3]:
rdrs_vars = [4, 5]
agg = None #['sum', 'minmax']
seq_len = 5*24
seq_steps = 1
validation_fraction = 0.1
batch_size = 32

train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(hours=seq_len * seq_steps)
train_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'

In [4]:
train_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, train_start, train_end, station=True, aggregate_daily=agg)
test_dataset = datasets.RdrsDataset(rdrs_vars, seq_len, seq_steps, test_start, test_end, station=True, aggregate_daily=agg,
                                    conv_scalers=train_dataset.conv_scalers, fc_scalers=train_dataset.fc_scalers)

val_indices = np.random.choice(len(train_dataset), size=int(validation_fraction * len(train_dataset)), replace=False)
train_indices = list(i for i in range(len(train_dataset)) if i not in val_indices)
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=train_sampler, pin_memory=True, drop_last=False)
val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, sampler=val_sampler, pin_memory=True, drop_last=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)

In [5]:
class LSTMRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, batch_size, dropout):
        super(LSTMRegression, self).__init__()
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout)
        self.linear = nn.Linear(hidden_dim, 1)
        self.init_hidden(batch_size)
    def init_hidden(self, batch_size):
        self.hidden = (torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device, requires_grad=True),
                       torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device, requires_grad=True))

    def forward(self, input):
        lstm_out, self.hidden = self.lstm(input.permute(1,0,2), self.hidden)
        return self.linear(lstm_out[-1])

In [6]:
# Create a mask of the grid that contains all grid cells that fall into a station's subwatershed
station_cell_mask = torch.zeros(train_dataset.x_conv.shape[-2:]).bool()
station_cell_mapping = load_data.get_station_cell_mapping()
for station in station_cell_mapping['station'].unique():    
    for _, row in station_cell_mapping[station_cell_mapping['station'] == station].iterrows():
        station_cell_mask[row['col'] - 1, row['row'] - 1] = True

onehot_vars = list(i for i,v in enumerate(train_dataset.fc_var_names) if v.startswith('station') or v.startswith('month'))

In [7]:
num_epochs = 1000
learning_rate = 2e-3
patience = 400
min_improvement = 0.01
H = 128
lstm_layers = 2
dropout = 0.3
weight_decay = 1e-5
best_loss_model = (-1, np.inf, None)
input_dim = train_dataset.x_conv.shape[2] * int(station_cell_mask.sum()) + len(onehot_vars)
model = LSTMRegression(input_dim, H, lstm_layers, batch_size, dropout).to(device)
loss_fn = evaluate.NSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

param_description = {'time_stamp': time_stamp, 'H': H, 'batch_size': batch_size, 'lstm_layers': lstm_layers, 'loss': loss_fn, 'optimizer': optimizer, 'lr': learning_rate, 
                     'patience': patience, 'min_improvement': min_improvement, 'dropout': dropout, 'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 
                     'train_start': train_start, 'train_end': train_end, 'weight_decay': weight_decay, 'validation_fraction': validation_fraction, 'test_start': test_start, 
                     'test_end': test_end, 'input_dim': input_dim, 'model': str(model).replace('\n','').replace(' ', ''), 'train len':len(train_dataset), 
                     'test len': len(test_dataset), 'rdrs_vars': rdrs_vars, 'aggregate_daily': agg}
writer = SummaryWriter()
writer.add_text('Parameter Description', str(param_description))
str(param_description)

"{'time_stamp': '20190905-102708', 'H': 128, 'batch_size': 32, 'lstm_layers': 2, 'loss': NSELoss(), 'optimizer': Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    eps: 1e-08\n    lr: 0.002\n    weight_decay: 1e-05\n), 'lr': 0.002, 'patience': 400, 'min_improvement': 0.01, 'dropout': 0.3, 'num_epochs': 1000, 'seq_len': 120, 'seq_steps': 1, 'train_start': datetime.datetime(2010, 1, 6, 0, 0), 'train_end': '2012-12-31', 'weight_decay': 1e-05, 'validation_fraction': 0.1, 'test_start': '2013-01-01', 'test_end': '2014-12-31', 'input_dim': 784, 'model': 'LSTMRegression((lstm):LSTM(784,128,num_layers=2,dropout=0.3)(linear):Linear(in_features=128,out_features=1,bias=True))', 'train len': 49113, 'test len': 33580, 'rdrs_vars': [4, 5], 'aggregate_daily': None}"

In [8]:
model = torch.load('../../pickle/models/LSTM_VIC-oneModel_allStations_20190905-102708.pkl', map_location='cpu')

In [9]:
#print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
#model.load_state_dict(best_loss_model[2])
model.eval()
predict = test_dataset.data_runoff.copy()
predict['actual'] = predict['runoff']
predict['runoff'] = np.nan
pred_array = np.array([])
for i, test_batch in enumerate(test_dataloader):
    x_test = test_batch['x_conv'][...,station_cell_mask].reshape(*test_batch['x_conv'].shape[:2], -1)
    x_test = torch.cat([x_test, test_batch['x_fc'][:,onehot_vars].unsqueeze(dim=1).repeat(1,x_test.shape[1],1)], dim=2).to(device)
    model.init_hidden(x_test.shape[0])
    pred_array = np.concatenate([pred_array, model(x_test).detach().cpu().numpy().reshape(-1)])
    
predict['runoff'] = pred_array

In [10]:
nse_list = []
mse_list = []
grouped_predict = predict.groupby('station')
for station in grouped_predict.groups.keys():
    station_predict = grouped_predict.get_group(station).set_index('date')
    nse, mse = evaluate.evaluate_daily(station, station_predict[['runoff']], station_predict['actual'], writer=writer)
    nse_list.append(nse)
    mse_list.append(mse)
    
    print(station, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')

result_str = 'Median NSE (clipped to 0) {} / Min {} / Max {}'.format(np.median(nse_list), np.min(nse_list), np.max(nse_list))
print(result_str)
print('Median MSE (clipped to 0)', np.median(mse_list), '/ Min', np.min(mse_list), '/ Max', np.max(mse_list))
writer.add_text('Results', result_str)

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


02GA010 	NSE: -0.01487434590386405 	MSE: 483.86524294371355 (clipped to 0)
02GA018 	NSE: 0.0812810162385933 	MSE: 230.73919475927738 (clipped to 0)
02GA038 	NSE: 0.11653072595862213 	MSE: 145.3646710949108 (clipped to 0)
02GA047 	NSE: -0.30057416421048666 	MSE: 101.95204081067533 (clipped to 0)
02GB001 	NSE: -0.2337025223522986 	MSE: 9264.951975066868 (clipped to 0)
02GB007 	NSE: 0.31392503455299814 	MSE: 21.454946287251236 (clipped to 0)
02GC002 	NSE: 0.2170335998568238 	MSE: 100.61355842428154 (clipped to 0)
02GC007 	NSE: -0.23456837105123518 	MSE: 37.18685918090604 (clipped to 0)
02GC010 	NSE: 0.3104240262504604 	MSE: 40.99523633463642 (clipped to 0)
02GC018 	NSE: 0.2723961615127093 	MSE: 49.278280902924706 (clipped to 0)
02GC026 	NSE: 0.20934581604935942 	MSE: 127.1630893458375 (clipped to 0)
02GD004 	NSE: 0.2529861029456023 	MSE: 41.13288105401522 (clipped to 0)
02GE007 	NSE: 0.200096219550977 	MSE: 28.936566035819332 (clipped to 0)
02GG002 	NSE: 0.2141797289564379 	MSE: 189.52635

In [11]:
writer.close()

In [12]:
load_data.pickle_results('LSTM_VIC-oneModel', predict[['date', 'station', 'runoff', 'actual']].rename({'runoff': 'prediction'}, axis=1).reset_index(drop=True), time_stamp)

'LSTM_VIC-oneModel_20190905-102708.pkl'

In [13]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190906-133600'